In [8]:
!virtualenv bertenv

Using base prefix 'c:\\users\\benar\\anaconda3'
  No LICENSE.txt / LICENSE found in source
New python executable in C:\Users\benar\Desktop\project_data\bertenv\Scripts\python.exe
Installing setuptools, pip, wheel...
done.


In [1]:
# libraries
import os
import csv
import time
import tensorflow as tf
import bert
import pandas as pd
from NYT_parser import NYTArticle
from utilities import *

In [18]:
# get pathways
train_paths = data_setup.get_paths('train')
article = NYTArticle.from_file(os.path.join('data', train_paths[0][0]))

In [20]:
# set destination folder and make folders if needed
dest_folder = 'sample/train/'
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

In [4]:
# attempt a single file output
print("Writing", dest_folder+article.docid)
article.simple_csv_output(dest_folder)

Writing sample/train/739212


In [5]:
# attempt 50 article write minus filtered out
for path in train_paths[:50]:
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        print("Writing", dest_folder+article.docid)
        article.simple_csv_output(dest_folder)

Writing sample/train/1397928
Writing sample/train/1609775
Writing sample/train/297100
Writing sample/train/448007
Writing sample/train/1463610
Writing sample/train/590096
Writing sample/train/762936
Writing sample/train/1570877
Writing sample/train/1725268
Writing sample/train/1647864
Writing sample/train/1251662
Writing sample/train/551924
Writing sample/train/74723
Writing sample/train/1535917
Writing sample/train/744457
Writing sample/train/97129
Writing sample/train/899133
Writing sample/train/791668
Writing sample/train/779448
Writing sample/train/1287650
Writing sample/train/927546
Writing sample/train/1587613
Writing sample/train/638654
Writing sample/train/870914
Writing sample/train/800527
Writing sample/train/1584905
Writing sample/train/1287373
Writing sample/train/1337862
Writing sample/train/1646102
Writing sample/train/1363692
Writing sample/train/50986
Writing sample/train/1481311
Writing sample/train/1265869
Writing sample/train/199802
Writing sample/train/1000735
Writi

In [14]:
# write all filtered articles in train set
opened_article = 0
written_article = 0
start = time.time()
for path in train_paths[:10000]:
    opened_article += 1
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        written_article += 1
        article.simple_csv_output(dest_folder)
    if opened_article % 10000 == 0:
        print("Opened", opened_article,"articles. Output", written_article,"articles.")
print(f"\n... processed {opened_article} files in {time.time() - start} seconds")

Opened 10000 articles. Output 8341 articles.

... processed 10000 files in 4.4035398960113525 seconds


In [15]:
# build dataframe with articles
sample = 1000

headlines = []
bodies = []

for path in train_paths[:sample]:
    article = NYTArticle.from_file(os.path.join('data',path[0]))
    if article.pass_filters():
        article.simple_csv_output(dest_folder)
        headlines.append(article.print_hede[0])
        # join paragraphs for now, may want to do something else later
        bodies.append('\n'.join(article.paragraphs))

train_df = pd.DataFrame(data={'headline':headlines, 'body':bodies})
train_df.head()

,headline,body
0,"A Killer, Yes, but She's a Good Old Girl",Aileen Wuornos was big news 10 years ago. At a...
1,"How High's The Bidding, Mama?",THE blue Rolls Royce sat at the York Street cu...
2,Pacificorp reports earnings for Qtr to Sept 30,LEAD:\n*3*** COMPANY REPORTS **\n*3* Pacificor...
3,Stewart Set to Return,There is bad news for Oakland's competition in...
4,Taxing Online Sales,"To the Editor:\nRe ''So Many Online Sales, So ..."


## Universal Transformer

In [16]:
# Install deps
!pip install -q -U tensor2tensor
!pip install -q tensorflow matplotlib

In [46]:
# Imports we need.
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

# Setup some directories
DATA_DIR = os.path.expanduser("./t2t/data")
TMP_DIR = os.path.expanduser("./t2t/tmp")
TRAIN_DIR = os.path.expanduser("./t2t/train")
# checkpoint_dir = os.path.expanduser("./t2t/checkpoints")
tf.gfile.MakeDirs(DATA_DIR)
tf.gfile.MakeDirs(TMP_DIR)
tf.gfile.MakeDirs(TRAIN_DIR)
# tf.gfile.MakeDirs(checkpoint_dir)
gs_data_dir = "gs://tensor2tensor-data/"
# gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"

In [47]:
# problems.available()

In [48]:
# Fetch the problem
sumCNN_problem = problems.problem("summarize_cnn_dailymail32k")

# Copy the vocab file locally so we can encode inputs and decode model outputs
# All vocabs are stored on GCS
# vocab_name = "vocab.translate_ende_wmt32k.32768.subwords"
vocab_name = "vocab.summarize_cnn_dailymail32k.32768.subwords"
vocab_file = os.path.join(DATA_DIR, vocab_name)
!gsutil cp {vocab_file} {DATA_DIR}

# Get the encoders from the problem
encoders = sumCNN_problem.feature_encoders(DATA_DIR)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
  """Input str to features dict, ready for inference"""
  inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs}

def decode(integers):
  """List of ints to str"""
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return encoders["inputs"].decode(np.squeeze(integers))

CommandException: cp: "file://./t2t/data/vocab.summarize_cnn_dailymail32k.32768.subwords" and "file://./t2t/data/vocab.summarize_cnn_dailymail32k.32768.subwords" are the same file - abort.


In [49]:
# Generate and view the data
# This cell is commented out because WMT data generation can take hours

# sumCNN_problem.generate_data(DATA_DIR, TMP_DIR)
# example = tfe.Iterator(sumCNN_problem.dataset(Modes.TRAIN, DATA_DIR)).next()
# inputs = [int(x) for x in example["inputs"].numpy()] # Cast to ints.
# targets = [int(x) for x in example["targets"].numpy()] # Cast to ints.

# # Example inputs as int-tensor.
# print("Inputs, encoded:")
# print(inputs)
# print("Inputs, decoded:")
# # Example inputs as a sentence.
# print(decode(inputs))
# # Example targets as int-tensor.
# print("Targets, encoded:")
# print(targets)
# # Example targets as a sentence.
# print("Targets, decoded:")
# print(decode(targets))

In [50]:
# There are many models available in Tensor2Tensor
# NOTE THAT THERE IS A UNIVERSAL TRANSFORMER IN HERE
# registry.list_models()

In [70]:
# Create hparams and the model
model_name = "transformer"
hparams_set = "transformer_prepend"

hparams = trainer_lib.create_hparams(hparams_set, data_dir=DATA_DIR, problem_name="summarize_cnn_dailymail32k")
# data_dir=DATA_DIR

# NOTE: Only create the model once when restoring from a checkpoint; it's a
# Layer and so subsequent instantiations will have different variable scopes
# that will not match the checkpoint.
sumCNN_model = registry.model(model_name)(hparams, Modes.TRAIN)

# from tensor2tensor.utils.trainer_lib import create_hparams

# # Init Hparams object from T2T Problem
# hparams = create_hparams(hparams)

In [71]:
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment

# Initi Run COnfig for Model Training
RUN_CONFIG = create_run_config(
      model_dir=TRAIN_DIR, # Location of where model file is store
      model_name=sumCNN_model
      # More Params here in this fucntion for controling how noften to tave checkpoints and more. 
)

# Create Tensorflow Experiment Object
tensorflow_exp_fn = create_experiment(
        RUN_CONFIG,#run_config=RUN_CONFIG,
        hparams,#=hparams,
        model_name,#="transformer",
#         model_name=RUN_CONFIG.model_name, #sumCNN_model,
        sumCNN_problem,#problem_name=sumCNN_problem,
        data_dir,
        train_steps=4000, # Total number of train steps for all Epochs
        eval_steps=100 # Number of steps to perform for each evaluation
    )

# Kick off Training
tensorflow_exp_fn.train_and_evaluate()

W0714 11:27:31.293293 140699800704832 devices.py:141] Schedule=continuous_train_and_eval. Assuming that training is running on a single machine.


ValueError: Hyperparameter name is reserved: data_dir

In [7]:
# Copy the pretrained checkpoint locally
# ckpt_name = "transformer_ende_test"
# gs_ckpt = os.path.join(gs_ckpt_dir, ckpt_name)
# !gsutil -q cp -R {gs_ckpt} {checkpoint_dir}
# ckpt_path = tf.train.latest_checkpoint(os.path.join(checkpoint_dir, ckpt_name))
# ckpt_path

In [8]:
# What checkpoints are available
# gs://tensor2tensor-data/
#     gs_data_dir
# !gsutil ls {gs_ckpt_dir}
# !gsutil ls {gs_data_dir}
# !gsutil -q cp -R {'gs://tensor2tensor-data/vocab.cnndailymail.32768'} {checkpoint_dir}
# !gsutil -q cp -R {'gs://tensor2tensor-data/vocab.summarize_cnn_dailymail32k.32768.subwords'} {checkpoint_dir}

In [9]:
# Prepare for the training loop

# In Eager mode, opt.minimize must be passed a loss function wrapped with
# implicit_value_and_gradients
@tfe.implicit_value_and_gradients
def loss_fn(features):
  _, losses = sumCNN_model(features)
  return losses["training"]

# Setup the training data
BATCH_SIZE = 1
sumCNN_problem = problems.problem("summarize_cnn_dailymail32k")
sumCNN_dataset = sumCNN_problem.dataset(Modes.TRAIN, DATA_DIR)
sumCNN_dataset = sumCNN_dataset.repeat(None).batch(BATCH_SIZE)

optimizer = tf.train.AdamOptimizer()

W0714 10:13:20.801770 140699800704832 deprecation.py:323] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/data_generators/problem.py:680: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
W0714 10:13:20.822395 140699800704832 deprecation_wrapper.py:119] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/data_generators/text_problems.py:394: The name tf.VarLenFeature is deprecated. Please use tf.io.VarLenFeature instead.

W0714 10:13:20.823091 140699800704832 deprecation_wrapper.py:119] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/data_generators/problem.py:705: The name tf.FixedLenFeature is deprecated. Plea

In [10]:
# Train
NUM_STEPS = 50

for count, example in enumerate(tfe.Iterator(sumCNN_dataset)):
#   print(example["targets"])
#   example["targets"] = tf.reshape(example["targets"], [BATCH_SIZE, 1, 1, 1])  # Make it 4D.
  loss, gv = loss_fn(example)
  optimizer.apply_gradients(gv)

#   if count % 50 == 0:
#     print("Step: %d, Loss: %.3f" % (count, loss.numpy()))
  print("Step: %d, Loss: %.3f" % (count, loss.numpy()))
  if count >= NUM_STEPS:
    break

W0714 10:13:20.940759 140699800704832 deprecation.py:323] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/utils/t2t_model.py:1367: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0714 10:13:21.074679 140699800704832 deprecation.py:323] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/layers/common_attention.py:856: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0714 10:13:21.103854 140699800704832 deprecation.py:323] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/utils/expert_utils.py:621: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0714 10

Step: 0, Loss: 9.689
Step: 1, Loss: 9.052
Step: 2, Loss: 8.666
Step: 3, Loss: 8.542
Step: 4, Loss: 8.843
Step: 5, Loss: 8.326
Step: 6, Loss: 7.927
Step: 7, Loss: 7.998
Step: 8, Loss: 7.685
Step: 9, Loss: 7.948
Step: 10, Loss: 7.234
Step: 11, Loss: 7.449
Step: 12, Loss: 7.382
Step: 13, Loss: 8.433
Step: 14, Loss: 8.022
Step: 15, Loss: 8.135
Step: 16, Loss: 7.377
Step: 17, Loss: 8.066
Step: 18, Loss: 7.910
Step: 19, Loss: 7.960
Step: 20, Loss: 8.058
Step: 21, Loss: 7.906
Step: 22, Loss: 7.623
Step: 23, Loss: 8.129
Step: 24, Loss: 7.757
Step: 25, Loss: 7.583
Step: 26, Loss: 7.684
Step: 27, Loss: 7.694
Step: 28, Loss: 7.949
Step: 29, Loss: 7.888
Step: 30, Loss: 7.330
Step: 31, Loss: 7.357
Step: 32, Loss: 7.251
Step: 33, Loss: 7.663
Step: 34, Loss: 7.737
Step: 35, Loss: 7.651
Step: 36, Loss: 7.781
Step: 37, Loss: 7.489
Step: 38, Loss: 6.954
Step: 39, Loss: 7.404
Step: 40, Loss: 7.713
Step: 41, Loss: 7.808
Step: 42, Loss: 7.281
Step: 43, Loss: 7.764
Step: 44, Loss: 8.200
Step: 45, Loss: 7.33

In [11]:
# inputs = """LEAD: New estimates by the Congressional Budget Office show that the cost of Government benefit programs is increasing much faster than had been expected.', 'New estimates by the Congressional Budget Office show that the cost of Government benefit programs is increasing much faster than had been expected.', 'The increase appears to result in part from decisions by state officials to expand social welfare programs such as Medicaid, which provides health care to the poor and is financed jointly by the Federal Government and the states.', 'In the Medicaid program, states set eligibility criteria and define the scope of benefits within guidelines set by Federal law. In the last year many states have expanded Medicaid eligibility or increased benefits to provide more assistance to pregnant women and children from low-income families.', 'In each state, the Federal Government pays at least half the costs of Medicaid and Aid to Families with Dependent Children, the main Federal-state welfare program. Giant Share of U.S. Spending', 'Benefit programs account for nearly half of all Federal spending. In August the budget office estimated that the cost of these programs would rise from $467 billion in the current fiscal year, 1987, to $499 billion in 1988 and eventually to $593 billion in 1991.', 'Aides to Rudolph G. Penner, director of the Congressional Budget Office, said today that the new estimates were $10 billion to $15 billion higher for 1987 and for 1988, and more than $20 billion higher for 1991.', 'These figures suggest that it will be difficult for Congress to meet the deficit targets set in the budget-balancing law signed by President Reagan in December 1985. The targets are $144 billion for 1987 and $108 billion for 1988. Under the law, the budget is supposed to be balanced by 1991.', ""Mr. Reagan plans to submit his 1988 budget to Congress on Monday. His budget director, James C. Miller 3d, has said that the President's budget would show a deficit of no more than $108 billion in 1988. Estimates of the cost of benefit programs have not yet been made public, and the budget does not contain a figure combining all the benefit programs."", 'The latest estimates by the Congressional Budget Office mean that Democrats will be under new pressure to restrain the growth of domestic programs that they might otherwise want to expand. Democrats in the Senate and the House have begun drafting legislation to expand welfare, Medicare, the Federal health insurance program for the elderly and disabled, and Medicaid.', 'Under the major benefit programs, such as Social Security, student loans and unemployment compensation, people are entitled to benefits if they meet certain eligibility requirements set by law. In some of the programs, there is a test of financial need. But in others, people can obtain benefits regardless of their income or asssets.', 'Several of the programs provide assistance to farmers to support the prices of agricultural commodities such as corn, wheat, rice and cotton. Aides to Mr. Penner said the projected costs of these programs had risen substantially above the levels estimated in August: $19.7 billion for 1987 and $21.6 billion for 1988. But they said the new figures would not be disclosed for at least a week. Change in Revenue Estimates', ""Administration officials said President Reagan's budget would show revenue dramatically different from that estimated by Congress when it approved a major tax bill in September."", 'At that time, the Congressional Joint Committee on Taxation compared the new measure with prior law and concluded that it would generate $11 billion more in 1987, but $17 billion less in 1988.', ""But Mr. Reagan's budget, using estimates developed by the Treasury Department, says the new law will produce a bigger gain in 1987, $18 billion, and no loss at all in 1988."", 'David H. Brockway, chief of staff for the joint committee, said the panel had not changed its estimates.', ""The Treasury's figures have political implications because they would make it easier for the President and Congress to reduce the deficit in 1987 and 1988. But in later years the Treasury's estimates would have a different effect, because they show smaller gains in revenue, or more of a loss, than the joint committee's estimates."", 'Most state income-tax laws are linked to Federal tax law in some way. Thirty-five states are expected to gain revenue if they make no change in their own laws. Some of the states plan to use part of that money to expand social welfare benefits.', 'Such changes could require a further increase in Federal spending because the Federal Government shares the cost of several major social welfare programs. The Federal contribution ranges from 50 percent to 78 percent of the cost of benefits under Medicaid and Aid to Families with Dependent Children. Increase for Science Foundation', 'While proposing cuts in many domestic programs, Administration officials said Mr. Reagan would seek an increase of more than 15 percent in the budget of the National Science Foundation, which supports research in science and engineering. The agency has a budget of $1.6 billion this year.', ""The proposed increase is part of Mr. Reagan's overall effort to ''keep America competitive'' by maintaining its technological edge over other countries."", 'But budget officials said Mr. Reagan would ask Congress to slash the budget of the Corporation for Public Broadcasting, which supports public radio and television stations around the country. Congress has already voted to give the corporation $214 million in 1988, $228 million in 1989 and $254 million 1990.', ""Mr. Reagan wants to cut the corporation's budget to $170 million in 1988, $140 million in 1989 and $132 million in 1990, according to confidential budget documents.""]"""

In [12]:
# Restore and translate!
def translate(inputs):
  encoded_inputs = encode(inputs)
#   with tfe.restore_variables_on_create(ckpt_path):
#     model_output = translate_model.infer(encoded_inputs)["outputs"]
  model_output = sumCNN_model.infer(encoded_inputs)["outputs"]
  return decode(model_output)

inputs = "New estimates by the Congressional Budget Office show that the cost of Government benefit programs is increasing much faster than had been expected.', 'New estimates by the Congressional Budget Office show that the cost of Government benefit programs is increasing much faster than had been expected.', 'The increase appears to result in part from decisions by state officials to expand social welfare programs such as Medicaid, which provides health care to the poor and is financed jointly by the Federal Government and the states.', 'In the Medicaid program, states set eligibility criteria and define the scope of benefits within guidelines set by Federal law. In the last year many states have expanded Medicaid eligibility or increased benefits to provide more assistance to pregnant women and children from low-income families.', 'In each state, the Federal Government pays at least half the costs of Medicaid and Aid to Families with Dependent Children, the main Federal-state welfare program. Giant Share of U.S. Spending', 'Benefit programs account for nearly half of all Federal spending. In August the budget office estimated that the cost of these programs would rise from $467 billion in the current fiscal year, 1987, to $499 billion in 1988 and eventually to $593 billion in 1991.', 'Aides to Rudolph G. Penner, director of the Congressional Budget Office, said today that the new estimates were $10 billion to $15 billion higher for 1987 and for 1988, and more than $20 billion higher for 1991.', 'These figures suggest that it will be difficult for Congress to meet the deficit targets set in the budget-balancing law signed by President Reagan in December 1985. The targets are $144 billion for 1987 and $108 billion for 1988. Under the law, the budget is supposed to be balanced by 1991.', ""Mr. Reagan plans to submit his 1988 budget to Congress on Monday. His budget director, James C. Miller 3d, has said that the President's budget would show a deficit of no more than $108 billion in 1988. Estimates of the cost of benefit programs have not yet been made public, and the budget does not contain a figure combining all the benefit programs."", 'The latest estimates by the Congressional Budget Office mean that Democrats will be under new pressure to restrain the growth of domestic programs that they might otherwise want to expand. Democrats in the Senate and the House have begun drafting legislation to expand welfare, Medicare, the Federal health insurance program for the elderly and disabled, and Medicaid.', 'Under the major benefit programs, such as Social Security, student loans and unemployment compensation, people are entitled to benefits if they meet certain eligibility requirements set by law. In some of the programs, there is a test of financial need. But in others, people can obtain benefits regardless of their income or asssets.', 'Several of the programs provide assistance to farmers to support the prices of agricultural commodities such as corn, wheat, rice and cotton. Aides to Mr. Penner said the projected costs of these programs had risen substantially above the levels estimated in August: $19.7 billion for 1987 and $21.6 billion for 1988. But they said the new figures would not be disclosed for at least a week. Change in Revenue Estimates', ""Administration officials said President Reagan's budget would show revenue dramatically different from that estimated by Congress when it approved a major tax bill in September."", 'At that time, the Congressional Joint Committee on Taxation compared the new measure with prior law and concluded that it would generate $11 billion more in 1987, but $17 billion less in 1988.', ""But Mr. Reagan's budget, using estimates developed by the Treasury Department, says the new law will produce a bigger gain in 1987, $18 billion, and no loss at all in 1988."", 'David H. Brockway, chief of staff for the joint committee, said the panel had not changed its estimates.', ""The Treasury's figures have political implications because they would make it easier for the President and Congress to reduce the deficit in 1987 and 1988. But in later years the Treasury's estimates would have a different effect, because they show smaller gains in revenue, or more of a loss, than the joint committee's estimates."", 'Most state income-tax laws are linked to Federal tax law in some way. Thirty-five states are expected to gain revenue if they make no change in their own laws. Some of the states plan to use part of that money to expand social welfare benefits.', 'Such changes could require a further increase in Federal spending because the Federal Government shares the cost of several major social welfare programs. The Federal contribution ranges from 50 percent to 78 percent of the cost of benefits under Medicaid and Aid to Families with Dependent Children. Increase for Science Foundation', 'While proposing cuts in many domestic programs, Administration officials said Mr. Reagan would seek an increase of more than 15 percent in the budget of the National Science Foundation, which supports research in science and engineering. The agency has a budget of $1.6 billion this year.', ""The proposed increase is part of Mr. Reagan's overall effort to ''keep America competitive'' by maintaining its technological edge over other countries."", 'But budget officials said Mr. Reagan would ask Congress to slash the budget of the Corporation for Public Broadcasting, which supports public radio and television stations around the country. Congress has already voted to give the corporation $214 million in 1988, $228 million in 1989 and $254 million 1990.', ""Mr. Reagan wants to cut the corporation's budget to $170 million in 1988, $140 million in 1989 and $132 million in 1990, according to confidential budget documents."
outputs = translate(inputs)

print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs)

W0714 10:15:28.412641 140699800704832 deprecation.py:323] From /home/mark/anaconda3/lib/python3.7/site-packages/tensor2tensor/models/transformer.py:1148: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


Inputs: New estimates by the Congressional Budget Office show that the cost of Government benefit programs is increasing much faster than had been expected.', 'New estimates by the Congressional Budget Office show that the cost of Government benefit programs is increasing much faster than had been expected.', 'The increase appears to result in part from decisions by state officials to expand social welfare programs such as Medicaid, which provides health care to the poor and is financed jointly by the Federal Government and the states.', 'In the Medicaid program, states set eligibility criteria and define the scope of benefits within guidelines set by Federal law. In the last year many states have expanded Medicaid eligibility or increased benefits to provide more assistance to pregnant women and children from low-income families.', 'In each state, the Federal Government pays at least half the costs of Medicaid and Aid to Families with Dependent Children, the main Federal-state welfare